In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import pickle
import time
import dill
#http://www.newyorksocialdiary.com/party-pictures?page=1
#url = "http://www.newyorksocialdiary.com/party-pictures"
#response = requests.get(url, params={"limit": 1000, "offset": 0})
#response = requests.get(url,params={'page':3})
#print response.url
#print response.text[:200] + "..."

In [2]:

def scrap1(soup):
    parent_div = soup.select('div[class=view-content]')[0] #Find (at most) *one*
    a=parent_div.select('span[class=field-content]')
    #print a
    N=len(a)
    event=[a[i].text for i in range(N) if (i % 2)==0]
    date=[a[i].text for i in range(N) if (i % 2)==1]
    link=[a[i].select('a')[0]['href'] for i in range (N) if (i%2)==0]
    df=pd.DataFrame({'event':event,'date':date,'link':link})
    return (df)


df=pd.DataFrame(columns=['event','date','link'])
for page in range (27):
    response = requests.get("http://www.newyorksocialdiary.com/party-pictures",params={'page':page})
    soup = BeautifulSoup(response.text,"lxml")
    ans=scrap1(soup)
    df=pd.concat([df,ans])

#Updating index

df.date=pd.DatetimeIndex(df.date)
df.index=df.date

before=datetime.strptime("30/11/14", "%d/%m/%y")
df=df.ix[before:]
df

    


,date,event,link
date,,,
2014-11-24,2014-11-24,Gala Guests,/party-pictures/2014/gala-guests
2014-11-20,2014-11-20,Equal Justice,/party-pictures/2014/equal-justice
2014-11-18,2014-11-18,Celebrating the Treasures,/party-pictures/2014/celebrating-the-treasures
2014-11-17,2014-11-17,Associates and Friends,/party-pictures/2014/associates-and-friends
2014-11-13,2014-11-13,Michael's 25th,/party-pictures/2014/michaels-25th
2014-11-12,2014-11-12,New York Lifelines,/party-pictures/2014/new-york-lifelines
2014-11-10,2014-11-10,Legends and Leaders,/party-pictures/2014/legends-and-leaders
2014-11-06,2014-11-06,Fall Fun,/party-pictures/2014/fall-fun
2014-11-05,2014-11-05,Park Avenue Armory's 2014 Gala Masquerade,/party-pictures/2014/park-avenue-armorys-2014-...


In [4]:
links=["http://www.newyorksocialdiary.com" + df.link[i] for i in range(len(df))]

In [193]:
def caption_list3(url):
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    
    captions = soup.select('div[class=photocaption]')  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    
    captions = soup.select('font[size="1"]')  #First caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    #for fontfrag in soup.select('font[size="1"]'):
     #       if fontfrag.string is None:
    #            continue
     #       captions_list.append(fontfrag.string.strip())
    captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"}) #Second caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.findAll("td", {"class": "photocaption","scope":"row"}) #Third caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #captions_list.extend(captions_list)
    
    captions = soup.select('td[class=photocaption]') #Fourth caption type
    captions_list.extend([captions[i].text for i in range (len(captions))])
    #aptions_list.extend(captions_list)
    
    if len(captions_list)>0:
        #print url
        return captions_list
    else:
        print url
        return []

In [195]:
def caption_list (url): #Retrieves url, returns a list of photo captions
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    
    captions = soup.select('div[class=photocaption]')  #First caption type
    if len(captions)>0:
        captions_list=[captions[i].text for i in range (len(captions))]
    else:
        captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"}) #Second caption type
        if len(captions)>0:
            captions_list=[captions[i].text for i in range (len(captions))]
        else:
            captions = soup.findAll("td", {"class": "photocaption","scope":"row"}) #Third caption type
            if len(captions)>0:
                captions_list=[captions[i].text for i in range (len(captions))]
            else: 
                captions = soup.select('td[class=photocaption]') #Fourth caption type
                if len(captions)>0:
                    captions_list=[captions[i].text for i in range (len(captions))]
    
    if len(captions_list)>0:
        return captions_list
    else:
        print url
        return []
    

d=[] #the caption containers
counter=0
for url in links:
    time.sleep(0.1)
    if len(caption_list3(url))>0:
        counter+=1
        d.extend(caption_list3(url))
    
    if counter%50==0: #Printing and saving files
        print(counter)
        outfile=open( str(counter+1), "wb" )
        pickle.dump(d, outfile)
        outfile.close()
    #a=all_captions + caption_list(url)
outfile=open( "raw_captions2", "wb" )
pickle.dump(d, outfile)
outfile.close()  
#url="http://www.newyorksocialdiary.com/party-pictures/2014/public-celebrations"
##a=caption_list(url)
#a

http://www.newyorksocialdiary.com/nysd/partypictures
50
100
150
200
250
http://www.newyorksocialdiary.com/party-pictures/2012/simple-pleasures
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150


In [197]:
len(d)

137621

In [185]:
url="http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers"

print url
response = requests.get(url)
soup=BeautifulSoup(response.text)
#captions = soup.findAll("td", {"class": "photocaption","scope":"row"})
#captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"})
#captions = soup.select('div[class=photocaption]')
#captions = soup.select('td[class=photocaption]')
#captions=soup.findAll("div", {"class":["class1", "class2"]})
#caption_list=[captions[i].text for i in range (len(captions))]
d=caption_list3(url)
d

http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers
http://www.newyorksocialdiary.com/party-pictures/2014/opening-numbers


[u'Susan Burke, and Eleanor Sullivan']

In [5]:
outfile=open( "raw_captions", "r" )
d=pickle.load(outfile)
outfile.close()  


In [198]:
a=d[:]

In [199]:
#Removing empty and very long captions
a = filter(None, a)
a = [a[i] for i in range (len(a))  if len(a[i])<=250]
len(a)

117192

In [200]:
b=a[:]
#b

In [201]:
#Converting connection words to ",
for i, caption in enumerate(b):
    b[i]=caption.replace("and",",")
for i, caption in enumerate(b):
    b[i]=caption.replace(";",",")
for i, caption in enumerate(b):
    b[i]=caption.replace("&",",")
for i, caption in enumerate(b):
    b[i]=caption.replace("with",",")
for i, caption in enumerate(b):
    b[i]=caption.replace('"',",")
for i, caption in enumerate(b):
    b[i]=caption.replace('/',",")

In [202]:
#Replacing white space with ","
#for i, caption in enumerate(b):
#    b[i]=caption.replace(" ",",")
len(b)

117192

In [203]:
#Splitting into list of lists
for i, caption in enumerate(b):
    b[i]=caption.split(",")
#c=reduce(lambda x, y: x+y, b)
#c


In [204]:
e=b[:]


In [205]:
#removing private names only and some other one-word trash
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        if len(person.split())<=1:
            #print i,j
            e[i][j]="xxxxxx"


In [206]:
#removing white spaces on both sides
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        e[i][j]=person.strip()


In [207]:
#removing caption with only one person
#r=e[:]

for i, caption in enumerate(e):
    if len(caption)<=1:
        e[i]=['xxxxxx']
        #del(e[i])


In [208]:
#removing xxxxxxx externally
g=e[:]
while ['xxxxxx'] in g:
    g.remove(['xxxxxx'])


In [24]:
#removing xxxxxx from within the lists
h=g[:]
for i, captions in enumerate(h):
    while 'xxxxxx' in captions:
        g[i].remove('xxxxxx')


In [209]:
unique_names=reduce(lambda x, y: x+y, h)
unique_names=list(set(unique_names))
len(unique_names)

76310

In [304]:
h

[[u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  u'Ron Iervolino'],
 [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  u'Lindsay Macbeth'],
 [u'Tom Murro'],
 [u'Ron Iervolino', u'Trish Iervolino', u'Russ Middleton', u'Lisa Middleton'],
 [u'Barbara Loughlin', u'Dr. Gerald Loughlin', u'Debbie Gelston'],
 [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  u'Nancy Brown'],
 [u'Bill Mack', u'Les Lieberman'],
 [u'Dr. David Lyden', u'Patricia Sorenson'],
 [u'Jimmy Cayne', u'Vince Tese', u'Pat Cayne'],
 [u'Stuart Oran', u'Les Lieberman', u'Hilary Oran'],
 [u'Vince Tese', u'Chuck Grodin'],
 [u'Dwight Gooden', u'Les Lieberman'],
 [u'Dr. Amy Cunningham-Bussel', u'Ray Mirra', u'Dr. Tyler Janovitz'],
 [u'Dan Shedrick', u'Samara Heafitz'],
 [u'Jason Adelman'],
 [u'Bart Scott', u'Mark Lapl'],
 [u'Mitch Rubin',

In [211]:
#creating caption dictionary
d1={}
for key,value in enumerate(h):
    d1[key]=value


In [212]:
d1

{0: [u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 1: [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  u'Ron Iervolino'],
 2: [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  u'Lindsay Macbeth'],
 3: [u'Tom Murro'],
 4: [u'Ron Iervolino',
  u'Trish Iervolino',
  u'Russ Middleton',
  u'Lisa Middleton'],
 5: [u'Barbara Loughlin', u'Dr. Gerald Loughlin', u'Debbie Gelston'],
 6: [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  u'Nancy Brown'],
 7: [u'Bill Mack', u'Les Lieberman'],
 8: [u'Dr. David Lyden', u'Patricia Sorenson'],
 9: [u'Jimmy Cayne', u'Vince Tese', u'Pat Cayne'],
 10: [u'Stuart Oran', u'Les Lieberman', u'Hilary Oran'],
 11: [u'Vince Tese', u'Chuck Grodin'],
 12: [u'Dwight Gooden', u'Les Lieberman'],
 13: [u'Dr. Amy Cunningham-Bussel', u'Ray Mirra', u'Dr. Tyler Janovitz'],
 14: [u'Dan Shedrick', u'Samara Heafitz'],
 15: [u'Jason 

In [216]:
len(e)

117192

In [214]:
pp={}
ppf={}
count=0
for p in range(len(d1)):
    name=unique_names[p]
    pp[name]=[]
    ppf[name]=[]
    #print name
    for i in range(len (d1)):
        if name in d1[i]:
            friends=d1[i]
            #print d[i]
            ppf[name].extend(friends)
    pp[name]=set(ppf[name])








In [224]:
#for i in len(pp):
pp1={}
for key in pp.keys():
    pp1[key]=(len(pp[key])-1)*2


In [225]:
from operator import itemgetter
pp2=sorted(pp1.iteritems(), key=itemgetter(1), reverse=True)
pp2[:100]

[(u'Jean Shafiroff', 584),
 (u'Mark Gilbertson', 542),
 (u'Gillian Miniter', 356),
 (u'Geoffrey Bradfield', 318),
 (u'Alina Cho', 300),
 (u'Somers Farkas', 284),
 (u'Kamie Lightburn', 280),
 (u'Andrew Saffir', 268),
 (u'Lucia Hwong Gordon', 264),
 (u'Yaz Hern', 262),
 (u'Debbie Bancroft', 258),
 (u'Sharon Bush', 256),
 (u'Lydia Fenet', 248),
 (u'Eleanora Kennedy', 244),
 (u'Bettina Zilkha', 224),
 (u'Kipton Cronkite', 224),
 (u'Jamee Gregory', 222),
 (u'Muffie Potter Aston', 218),
 (u'Karen LeFrak', 212),
 (u'Audrey Gruss', 212),
 (u'Deborah Norville', 208),
 (u'Leonard Lauder', 208),
 (u'Adelina Wong Ettelson', 196),
 (u'Liz Peek', 196),
 (u'Barbara Tober', 196),
 (u'Amy Fine Collins', 190),
 (u'Lisa Anastos', 190),
 (u'Grace Meigher', 182),
 (u'Nicole Miller', 182),
 (u'a friend', 182),
 (u'Fern Mallis', 180),
 (u'Martha Glass', 178),
 (u'Dayssi Olarte de Kanavos', 178),
 (u'Margaret Russell', 176),
 (u'Karen Klopp', 176),
 (u'Nathalie Kaplan', 174),
 (u'a Kellogg', 170),
 (u'Wendy C

In [227]:
pd.Series([i[1] for i in pp2]).describe()

count    58737.000000
mean         6.968589
std         11.717051
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        584.000000
dtype: float64

In [228]:
def bff(friends):
    d2={}
    unique_friends=list(set(friends))
    for name in unique_friends:
          d2[name]=friends.count(name)
    return d2


In [305]:
bff_dict={}
for name in ppf.keys():
    bff_dict[name]=bff(ppf[name])
bff_dict

{u'Tom Smarr': {u'Barbara Paul Robinson': 1,
  u'Claudia Bonn': 1,
  u'Louis Bauer': 1,
  u'Marechal Brown': 1,
  u'Mark Fisher': 1,
  u'Tom Smarr': 1},
 u'Christie Fraser': {u'Christie Fraser': 1, u'Liz Celeste': 1},
 u'Dennis Serras': {u'Dennis Serras': 1,
  u'Marlo Bakst': 1,
  u'Mikhail Baryshnikov': 1,
  u'Tara Nasta': 1},
 u'Honorees: Richard C. Hunt': {u'Andy Jagoda': 1,
  u'Honorees: Richard C. Hunt': 1,
  u'Kevin W. Chason': 1,
  u'Susan C. Waltman': 1},
 u'Samuel Barnett': {u'Jano Herbosch': 1,
  u'Joseph Timms': 1,
  u'Samuel Barnett': 1},
 u'Jennifer Lantzas': {u'Ashley Ludwig': 1,
  u'Jennifer Lantzas': 1,
  u'Jonathan Kuhn': 1,
  u'Will Ryman': 1},
 u'Pearson Chodak': {u'Jonathan Sheffer': 1, u'Pearson Chodak': 1},
 u'Sarah Julig': {u'Sarah Julig': 1, u'Tom Finkelpearl': 1},
 u'Jackie Terrebonne': {u'Jackie Terrebonne': 1,
  u'Jolene Travis': 1,
  u'Lee Cavanaugh': 1},
 u'Jaime E. Jim\xe9nez of Baccarat': {u'Chair Clare McKeon': 1,
  u'Chair Sloan Overstrom': 1,
  u'Jaime

In [306]:
ans3={}
for person in bff_dict.keys():
    for friend in bff_dict[person].keys():
            if person!=friend:
                ans3[person+','+friend]=bff_dict[person][friend] * 2
ans3

{u'Bob Wankel,Joe Benincasa': 2,
 u'Betty Gorab,Esra Munnell': 2,
 u'Astrid Misrahi,Gala Co-Chair Judy Zankel': 2,
 u'Dylan Walsh,Bryan Batt': 2,
 u'Noah Chesnin,Barbara Pryor': 2,
 u'Anita Contini,Gala Co-Chair Frances Resheske': 2,
 u'Tim Daly,Michael Frankfurt': 2,
 u'Danielle Smotrich,Yigal Azrouel': 2,
 u'Laurie Malman,Nancy Moonves': 2,
 u'Balasz Nyari,Francine Nyari': 2,
 u'Gennaro Vitally,Theresa Beasley': 2,
 u'Lydia Fenet,Mary Giuliani': 6,
 u'Ricardo Vollbrechthausen,Adrian Deseglise': 2,
 u'Dr. Feder,Daniella Nik': 2,
 u'Kim Bates,Victoria Holloway': 2,
 u'Nikki M. James,Joel Silverman': 2,
 u'Dr. Bonnie M. Davis,Peter W. May': 2,
 u'Maureen Robertson,Rachel Leonard': 2,
 u'Gladys George,Dr. Stuart Orsher': 6,
 u'Leigh Bishop Taub,Katie Holmes.': 2,
 u'Cold Pina Coladas,Francois Payard': 2,
 u'Elena Phillips,Sarah Ramsey': 2,
 u'John W. Munson,Eiseley Tauginas': 2,
 u'Judy Libe,Emmy Lou Kelly': 2,
 u'Charlotte Ford,Dr. Steven J. Corwin': 2,
 u"Natalie Pfanstiehl is a well-k

In [302]:
from operator import itemgetter
ans4=sorted(ans3.iteritems(), key=itemgetter(1), reverse=True)
ans4[:100]
list(ans4)
x=ans4
for i in range (200):
    if (i % 2 ==0):
        x=ans4[i]
        print str(tuple([tuple(x[0].split(',')),x[1]])) + ","

#print tuple([tuple(x[0][0].split(',')),x[0][1]])
#ans4[0][1]
#tuple(x) for x in 
pd.Series([i[1] for i in ans4[:100]]).describe()

((u'Bonnie Comley', u'Stewart Lane'), 108),
((u'Andrew Saffir', u'Daniel Benedict'), 104),
((u'Roric Tobin', u'Geoffrey Bradfield'), 102),
((u'Richard Johnson', u'Sessa von Richthofen'), 54),
((u'Karl Wellner', u'Deborah Norville'), 52),
((u'Wilbur Ross', u'Hilary Geary Ross'), 50),
((u'Guy Robinson', u'Elizabeth Stribling'), 50),
((u'Elizabeth Stribling', u'Guy Robinson'), 50),
((u'Douglas Hannant', u'Frederick Anderson'), 48),
((u'Olivia Palermo', u'Johannes Huebl'), 46),
((u'Marc Rosen', u'Arlene Dahl'), 44),
((u'Anne Hearst McInerney', u'Jay McInerney'), 42),
((u'Leonel Piraino', u'Nina Griscom'), 38),
((u'Gillian Miniter', u'Sylvester Miniter'), 36),
((u'John Catsimatidis', u'Margo Catsimatidis'), 34),
((u'Brian Stewart', u'Stephanie Krieger'), 34),
((u'Stephanie Krieger', u'Brian Stewart'), 34),
((u'David Lauren', u'Lauren Bush'), 32),
((u'Jean Shafiroff', u'Sharon Bush'), 32),
((u'ra of Greece', u'Princess Alex'), 30),
((u'Michael Cominotto', u'Dennis Basso'), 30),
((u'Tony Ingr

count    100.000000
mean      34.580000
std       19.282682
min       20.000000
25%       24.000000
50%       26.000000
75%       36.500000
max      108.000000
dtype: float64

In [309]:
import networkx as nx
G=nx.Graph()
for i in ans4:
    edges=i[0].split(',')
    G.add_edge(edges[0],edges[1])
len(G.edges())
#ans4[1][0].split(':')


125803

(u'Bonnie Comley,Stewart Lane', 108)

In [310]:
prank=nx.pagerank(G)

In [311]:
prank1=sorted(prank.iteritems(), key=itemgetter(1), reverse=True)
prank1[:100]
#pd.Series([i[1] for i in prank1[:100]]).describe()

[(u'Jean Shafiroff', 0.0005883301448519918),
 (u'Mark Gilbertson', 0.0005564620236412059),
 (u'Gillian Miniter', 0.00035857918817658557),
 (u'Geoffrey Bradfield', 0.00033816087918971436),
 (u'Andrew Saffir', 0.00032573952672563866),
 (u'Alina Cho', 0.00030544771474254535),
 (u'ra Lebenthal', 0.0002909504858577131),
 (u'Sharon Bush', 0.000288783615805176),
 (u'Somers Farkas', 0.00028793182795587997),
 (u'Kamie Lightburn', 0.0002789542086078787),
 (u'Debbie Bancroft', 0.0002713717818618791),
 (u'Lucia Hwong Gordon', 0.0002660127671674815),
 (u'Yaz Hern', 0.0002616807923747062),
 (u'Mario Buatta', 0.0002614017648468655),
 (u'a friend', 0.00025669912976929637),
 (u'Lydia Fenet', 0.00025636735319167954),
 (u'Kipton Cronkite', 0.00025135069193752915),
 (u'Leonard Lauder', 0.0002393165907033093),
 (u'Eleanora Kennedy', 0.0002332938082278132),
 (u'Barbara Tober', 0.000233105104787836),
 (u'Bettina Zilkha', 0.00022316019023237442),
 (u'Patrick McMullan', 0.00022151841656354172),
 (u'Jamee Grego